In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import random
import re
import datasets
import tqdm
import math
from functools import partial
import math
import argparse
import os
import collections
import json
import sentencepiece
import shutil
import copy
import multiprocessing


torch.set_float32_matmul_precision("high")

In [ ]:
embedding_training_args = {
    "batch_size": 256,  # CPU용으로 축소 (32768 → 256)
    "epochs": 10,
    "lr": 1e-3,
    "device": "cpu",  # RTX 5070 CUDA 호환성 문제로 CPU 사용
    "embedding_dim": 384,

    "vocab_size": 50000,
    "min_freq": 5,

    "gradient_accumulate_steps": 1,
}

tokenizer_args = {
    "vocab_size": 50000,
    "min_freq": 5,
}

In [23]:
'''
토크나이저 : 텍스트를 모델이 이해할 수 있게 토큰 단위로 나누는 과정
BPE : 자주 등장하는 문자 조합을 하나의 토큰으로 묶는다
special_tokens 로는 전체 어휘 구성시 사용
additional_special_tokens 는 추가 토큰만 사용시
'''
import re
class Tokenizer:
    def __init__(self, 
                 max_vocab_size=10000, #최대 어휘 크기 (단어 개수 제한)
                 special_tokens=[], #추가 특수 토큰 리스트
                 pad_token="<|PAD|>", #패딩 (시퀀스 길이 맞춤용)
                 unk_token="<|UNK|>", #미등록 단어
                 bos_token="<|BOS|>", #문장 시작
                 eos_token="<|EOS|>"): #문장 종료
        self.max_vocab_size = max_vocab_size #최대 어휘 크기 저장
        self.tokenizer = None #토크나이저 객체 (하위 클래스에서 사용)

        self.pad_token = pad_token
        self.pad_token_id = 0
        self.unk_token = unk_token
        self.unk_token_id = 1
        self.bos_token = bos_token
        self.bos_token_id = 2
        self.eos_token = eos_token
        self.eos_token_id = 3

        self.special_tokens = [self.pad_token, self.unk_token, self.bos_token, self.eos_token] + special_tokens
        self.additional_special_tokens = special_tokens
    
    def save(self, path): #저장경로 디렉터리 생성
        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path,"mics.json"), "w") as f: #mics.json 생성
            json.dump({ #메타 정보 json 으로 지정 후 저장
                "max_vocab_size": self.max_vocab_size,
                "special_tokens": self.special_tokens,
                "additional_special_tokens": self.additional_special_tokens,
            }, f)
    
    def load(self, path):
        with open(os.path.join(path,"mics.json"), "r") as f: #디렉토리에서 load
            mics = json.load(f)
            self.max_vocab_size = mics["max_vocab_size"]
            self.special_tokens = mics["special_tokens"]
            self.additional_special_tokens = mics["additional_special_tokens"] #읽어오기

class WitespaceTokenizer(Tokenizer): #기본 상속
    def __init__(self, min_count=5, max_vocab_size=10000, special_tokens=[]):
        super().__init__(max_vocab_size, special_tokens) #부모 클래스 초기화
        self.min_count = min_count #최소 등장 횟수 제한
        self.vocab = {} #어휘 사전 초기화

        assert len(special_tokens) == len(set(special_tokens)), "Duplicate special tokens are not allowed." #중복 토큰 확인 만약 중복 있으면 오류 메시지
        assert len(special_tokens) < max_vocab_size, "Special tokens exceed max vocab size." #토큰 개수 제한

    def __len__(self):
        return len(self.vocab) #어휘 사전 크기 반환

    def normalize(self, text):
        import re  # multiprocessing 호환을 위해 함수 내부에서 import
        return re.sub(r'[^\w\s]', '', text).lower() #문자열 정규화

    def fit(self, dataset): #어휘 사전 구축
        token_counter = Counter() #단어 빈도수 카운터
        for text in tqdm.tqdm(dataset, desc="WitespaceTokenizer fitting..."): #진행상황 표시시
            tokens = self.normalize(text).split() #정규화 후 공백 분리
            token_counter.update(tokens) #토큰 빈도수 업데이트
        
        vocab = [(word,count) for word, count in token_counter.items() if count >= self.min_count] #vocab 구축
        vocab = sorted(vocab, key=lambda x: -x[1]) #빈도수 내림차순 정렬
        vocab = vocab[:(self.max_vocab_size - len(self.special_tokens))] #특수 토큰 개수 빼서 최대 어휘 크기 맞춤
        vocab = [word for word, _ in vocab] #단어 리스트로 변환
        vocab = self.special_tokens + vocab #특수 토큰 추가
        
        self.sample_weights = [] #샘플 가중치 초기화
        for i, word in enumerate(vocab): #리스트를 순회하면서 인덱스와 값 동시 반환
            self.vocab[word] = i
            self.sample_weights.append(math.log(token_counter[word]) if word in token_counter else -987654321) #빈도수 로그 취해서 가중치 계산

    def tokenize(self, text, add_special_tokens=False): #토큰화
        tokens = self.normalize(text).split()
        if add_special_tokens:
            tokens = ["<|BOS|>"] + tokens + ["<|EOS|>"]
        return [self.vocab.get(token, self.vocab["<|UNK|>"]) for token in tokens] #get(키, 기본값) 키가 없으면 기본값 반환 & 모든 unk 는 동일 id

    def save(self, path):
        super().save(path)
        with open(os.path.join(path,"word2idx.json"), "w") as f:
            json.dump(self.vocab, f)
        with open(os.path.join(path,"sample_weights.json"), "w") as f:
            json.dump(self.sample_weights, f)
        
    
    def load(self, path):
        super().load(path)
        with open(os.path.join(path,"word2idx.json"), "r") as f:
            self.vocab = json.load(f)
        with open(os.path.join(path,"sample_weights.json"), "r") as f:
            self.sample_weights = json.load(f)

'''
SentencePiece 의 주요 내장 메서드
get_piece_size()	전체 어휘 크기 반환
encode(text)	텍스트 → 토큰 ID 리스트
decode(ids)	토큰 ID 리스트 → 텍스트
bos_id()	BOS 토큰 ID
eos_id()	EOS 토큰 ID
load(path)	모델 파일 로드
'''
class BPETokenizer(Tokenizer):
    def __init__(self, max_vocab_size=10000, special_tokens=[]):
        super().__init__(max_vocab_size, special_tokens)
        self.bpe = None #SentencePiece BPE 모델 저장할 변수
        self.sample_weights = None #토큰별 가중치 저장 예정
        assert len(special_tokens) == len(set(special_tokens)), "Duplicate special tokens are not allowed." #중복 토큰 확인 없으면 에러 메시지
        assert len(special_tokens) < max_vocab_size, "Special tokens exceed max vocab size." #토큰 개수 제한 만약 넘어가면 에러 메세지
    
    def __len__(self):
        return self.bpe.get_piece_size() #전체 어휘 크기 반환

    def fit(self,dataset, save_path):
        print("Training BPE Tokenizer...")
        os.makedirs(save_path, exist_ok=True)
        prefix = os.path.join(save_path,"bpe")
        sentencepiece.SentencePieceTrainer.train(
            sentence_iterator=iter(dataset), #학습 데이터
            model_prefix=prefix, #모델 저장 경로
            vocab_size=self.max_vocab_size, #최대 어휘 크기
            max_sentence_length=100000, #최대 문장 길이
            shuffle_input_sentence=False, #입력 순서 유지
            byte_fallback=True, #unknown 문자를 byte 로 처리, 즉 안녕하세요 라는 oov 도 [<0XA9>,<0XW8>...] 이런식으로 처리 가능하다는 말
            num_threads=32,
            pad_id=0,
            pad_piece="<|PAD|>",
            unk_id=1,
            unk_piece="<|UNK|>",
            bos_id=2,
            bos_piece="<|BOS|>",
            eos_id=3,
            eos_piece="<|EOS|>",
            user_defined_symbols=self.additional_special_tokens
        )
        self.bpe = sentencepiece.SentencePieceProcessor() #모델 객체 생성
        self.bpe.load(prefix + ".model")
        with open(prefix+".vocab", "r", encoding="utf-8") as f:
            self.vocab = {}
            self.sample_weights = []
            for l in f:
                token, weight = l.strip().split("\t")
                self.vocab[token] = len(self.vocab)
                self.sample_weights.append(weight if token not in self.special_tokens else -987654321)

    def tokenize(self, text, add_special_tokens=False):
        tokens = self.bpe.encode(text, out_type=int)
        if add_special_tokens:
            tokens = [self.bpe.bos_id] + tokens + [self.bpe.eos_id]
        return tokens

    def save(self, path):
        super().save(path)
    
    def load(self, path):
        super().load(path)
        self.bpe = sentencepiece.SentencePieceProcessor()
        self.bpe.load(os.path.join(path,"bpe.model"))
        with open(os.path.join(path,"bpe.vocab"), "r", encoding="utf-8") as f:
            self.vocab = {}
            self.sample_weights = []
            for l in f:
                token, weight = l.strip().split("\t")
                self.vocab[token] = len(self.vocab)
                self.sample_weights.append(weight)

In [ ]:
dataset = datasets.load_dataset("abisee/cnn_dailymail",'3.0.0')
corpus_train_dataset = dataset['train'].select(range(50000))
corpus_vaildation_dataset = dataset['validation']

if os.path.exists("output/whitespace_tokenizer"):
    white_space_tokenizer = WitespaceTokenizer()
    white_space_tokenizer.load("output/whitespace_tokenizer")
else:
    white_space_tokenizer = WitespaceTokenizer(tokenizer_args['min_freq'],tokenizer_args['vocab_size'])
    white_space_tokenizer.fit(corpus_train_dataset['article'])
    white_space_tokenizer.save("output/whitespace_tokenizer")

if os.path.exists("output/bpe_tokenizer"):
    bpe_tokenizer = BPETokenizer()
    bpe_tokenizer.load("output/bpe_tokenizer")
else:
    bpe_tokenizer = BPETokenizer(tokenizer_args['vocab_size'])
    bpe_tokenizer.fit(corpus_train_dataset['article'], "output/bpe_tokenizer")
    bpe_tokenizer.save("output/bpe_tokenizer")

In [25]:
'''
중심 단어로 주변 단어 예측
네거티브 샘플링 추가 즉 상관 없는 단어들 배치
examples['article'] 이 해당 데이터 칼럼에 text 가 들어가 있음
'''
def sliding_window_preprocess(examples, tokenizer, window_size=2, num_negative_samples=-1):
    contexts = [] #주변 단어들
    targets = [] #중심 단어
    negatives = [] #네거티브 샘플
    for example in examples['article']:
        tokens = tokenizer.tokenize(example)
        for i in range(window_size, len(tokens) - window_size):
            context = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            target = tokens[i]
            contexts.append(context)
            targets.append(target)
            
            if num_negative_samples > 0:
                sampling_weight = copy.deepcopy(tokenizer.sample_weights)
                for t in tokens[i-window_size:i+window_size+1]:
                    sampling_weight[t] = -987654321
                negatives.append(random.choices(range(len(tokenizer)), weights=sampling_weight, k=num_negative_samples))
            else:
                negatives.append([])
    return {
        "context": contexts,
        "target": targets,
        "negative": negatives,
    }
def sliding_window_collate_fn(batch):
    contexts = torch.LongTensor([k['context'] for k in batch])
    targets = torch.LongTensor([k['target'] for k in batch])
    negatives = torch.LongTensor([k['negative'] for k in batch]) if batch[0]['negative'] else None
    return {
        "context": contexts,
        "target": targets,
        "negative": negatives,
    }

sw_wt_train_dataset = corpus_train_dataset.map(sliding_window_preprocess,
                                 batched=True,
                                 num_proc=os.cpu_count()//2,
                                 remove_columns=corpus_train_dataset.column_names,
                                 fn_kwargs={"tokenizer": white_space_tokenizer, "window_size": 2, "num_negative_samples": -1})

sw_wt_valid_dataset = corpus_vaildation_dataset.map(sliding_window_preprocess,
                                 batched=True,
                                 num_proc=os.cpu_count()//2,
                                 remove_columns=corpus_vaildation_dataset.column_names,
                                 fn_kwargs={"tokenizer": white_space_tokenizer, "window_size": 2, "num_negative_samples": -1})


Epoch 1/10:   0%|          | 0/9370 [15:10<?, ?it/s]


## Module for SkipGram with Negative Sampling (nn.Module) -- Your codes are required 

In [26]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    
    def forward(self, context, target, negative):
        context_embeds = self.embeddings(context) #임베딩 들어가기
        context_mean = context_embeds.mean(dim=1) #평균 벡터 계산
        target_embed = self.embeddings(target) #중심 단어 임베딩
        if negative is not None:
            negative_embeds = self.embeddings(negative) #네거티브 샘플 임베딩
            positive_score = torch.sum(context_mean * target_embed, dim=1) #정답 예측
            positive_loss = -F.logsigmoid(positive_score)
            negative_score = torch.bmm(negative_embeds, context_mean.unsqueeze(2)).squeeze(2) #네거티브 샘플 예측
            negative_loss = -F.logsigmoid(-negative_score).sum(dim=1)
            loss = (positive_loss + negative_loss).mean()
        else:
            # Negative Sampling 없이 Cross-Entropy Loss 사용
            logits = torch.matmul(context_mean, self.embeddings.weight.T)
            loss = F.cross_entropy(logits, target)
            
        return loss
        
        ## YOUR CODES END

In [27]:
def train(model, train_dataset, valid_dataset, collate_fn, train_args, prefix):
    optimzier = optim.Adam(model.parameters(), lr=train_args["lr"])

    train_dataloader = DataLoader(train_dataset, batch_size=train_args['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0)  # Windows 호환
    valid_dataloader = DataLoader(valid_dataset, batch_size=train_args['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0)  # Windows 호환

    total_steps = len(train_dataloader) * train_args['epochs']

    best_loss = 987654321
    
    output_path = os.path.join("output", prefix)
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, "train_args.json"), "w") as f:
        json.dump(train_args, f)

    pbar = tqdm.tqdm(total=total_steps, desc="training")
    for epoch in range(train_args['epochs']):
        pbar.set_description(f"Epoch {epoch+1}/{train_args['epochs']}")
        move_avg_loss = []
        model.train()
        for i, batch in enumerate(train_dataloader):
            batch = {k:v.to(train_args['device']) if isinstance(v,torch.Tensor) else v for k,v in batch.items()}

            loss = model(**batch)
            loss = loss / train_args['gradient_accumulate_steps']
            if loss.size() != torch.Size([]):
                loss = loss.mean()
            loss.backward()
            
            if (i+1) % train_args['gradient_accumulate_steps'] == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
                optimzier.step()
                optimzier.zero_grad()

            move_avg_loss.append(loss.item()) 
            if len(move_avg_loss) > 100: move_avg_loss.pop(0)
            pbar.set_postfix_str(f"loss: {sum(move_avg_loss)/len(move_avg_loss):.04f} lr: {optimzier.param_groups[0]['lr']:.2e}")
            pbar.update(1)
        
        model.eval()
        with torch.no_grad():
            eval_loss = 0
            for i, batch in enumerate(valid_dataloader):
                with torch.no_grad():
                    batch = {k:v.to(train_args['device']) if isinstance(v,torch.Tensor) else v for k,v in batch.items()}
                    loss = model(**batch)
                    if loss.size() != torch.Size([]):
                        loss = loss.mean()
                    eval_loss += loss.item()
                    pbar.set_postfix_str(f"val_loss: {eval_loss / (i+1):.04f}")
        eval_loss /= len(valid_dataloader)
        pbar.write(f"Validation Loss: {eval_loss:.04f}")

        if eval_loss < best_loss:
            best_loss = eval_loss
            
            torch.save(model.state_dict(), os.path.join(output_path,"best_model.pth"))
            pbar.write(f"Model Saved best loss: {best_loss:.04f}")

    pbar.close()

In [ ]:
sg_wt_model = SkipGram(vocab_size=len(white_space_tokenizer), embedding_dim=embedding_training_args['embedding_dim'])
embedding_training_args['device'] = 'cpu'  # RTX 5070 CUDA 호환성 문제로 CPU 사용
sg_wt_model = sg_wt_model.to(embedding_training_args['device'])
sg_wt_model = nn.DataParallel(sg_wt_model) if torch.cuda.device_count() > 1 else sg_wt_model
# sg_wt_model = torch.compile(sg_wt_model)  # Windows 호환성 문제로 비활성화
train(sg_wt_model, sw_wt_train_dataset, sw_wt_valid_dataset, sliding_window_collate_fn, embedding_training_args, "sg_wt")
best_model_statedict = torch.load("output/sg_wt/best_model.pth")
sg_wt_model.load_state_dict(best_model_statedict)

Epoch 1/10:   1%|          | 7611/1199360 [31:01<150:58:09,  2.19it/s, loss: 25.1760 lr: 1.00e-03]